<a href="https://colab.research.google.com/github/fact-h/kaggle-sklearn-ML-tutorial/blob/main/Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines

## 介绍

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()